In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
!ls "/content/gdrive/My Drive/Research/ML for fluid dynamics/RS_AK_code"

__pycache__  solvers.py  TrainingData.ipynb  TrainingData_SD.ipynb


In [10]:
path = F"/content/gdrive/My Drive/Research/ML for fluid dynamics/RS_AK_code"

In [11]:
import sys
# Append the directory to your python path using sys
sys.path.append(path)

In [14]:
import numpy as np
import solvers
from plotly import graph_objs as go
from plotly.subplots import make_subplots
from tqdm.notebook import trange

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from mpl_toolkits import mplot3d

In [60]:
#Create a list of parameters to iterate through

#Global parameters:
L = 43
h = 1 # grid spacing
N_g = int(L/h)
N = 1 # Number of Particles
# N = 3 # Number of Particles

np.random.seed(42)

DatasetSize = 20000
#Local Parameters
LocalParameterArray = np.zeros((4, DatasetSize))
#Generating Sizes
# a_values = np.around(np.arange(0.1, 21.6,0.1), decimals = 1)
# LocalParameterArray[0] = np.random.choice(a_values, size = DatasetSize)
LocalParameterArray[0] = np.random.randint(7,20, size = DatasetSize)
# print(LocalParameterArray)

#Generating Positions
#r = [rx, ry, N_g/2]
# LocalParameterArray[2:] = np.random.randint(0, N_g, size = (2, DatasetSize))
LocalParameterArray[2:] = np.random.randint(13, 32, size = (2, DatasetSize))
# print(LocalParameterArray)

#Generating Force amplitude
# f_values = np.around(np.arange(0.1, 50.1,0.1), decimals = 1)
f_values = np.around(np.arange(1, 20, 0.2), decimals = 1)
LocalParameterArray[1] = np.random.choice(f_values, size = DatasetSize)
# print(LocalParameterArray)


parameters = {'h':h, 'Lx':L, 'Ly':L, 'Lz':L, 'N':N, 'N_g':N_g, 'indexingConvention':'ij' }
sim = solvers.Stokes3D(parameters)

In [62]:
# vImages = np.zeros((3*DatasetSize, int(1+(L-1)/3),int(1+(L-1)/3)))
# vFlatImages = np.zeros((3*DatasetSize, int(1+(L-1)/3)**2))
# fImages = np.zeros((3*DatasetSize, int(1+(L-1)/3),int(1+(L-1)/3)))
# fFlatImages = np.zeros((3*DatasetSize, int(1+(L-1)/3)**2))

vImages = np.zeros((DatasetSize, int(1+(L-1)/3),int(1+(L-1)/3)))
v_train = np.zeros((DatasetSize, int(1+(L-1)/3)*int(1+(L-1)/3)))
fImages = np.zeros((DatasetSize, int(1+(L-1)/3),int(1+(L-1)/3)))
f_train = np.zeros((DatasetSize, int(1+(L-1)/3)*int(1+(L-1)/3)))

for i in trange(DatasetSize):

    a = LocalParameterArray[0][i]
    par_pl = int(N_g/2)
    r = np.array((LocalParameterArray[2][i], LocalParameterArray[3][i], sim.xx[par_pl]))
    F = np.array((0,0, LocalParameterArray[1][i]))
    sigma = a/(np.sqrt(3))
    sim.vUpdate(a, sigma, r, F)

    #Turns vx, vy and vz from a scalar to an array
    vx = np.array((sim.vx[:, par_pl, :].reshape(N_g, N_g)))
    vy = np.array((sim.vy[:, par_pl, :].reshape(N_g, N_g)))
    vz = np.array((sim.vz[:, par_pl, :].reshape(N_g, N_g)))

    #Slices F_x, F_y and F_z
    F_x = np.array((sim.F_x[:, par_pl, :].reshape(N_g, N_g)))
    F_y = np.array((sim.F_y[:, par_pl, :].reshape(N_g, N_g)))
    F_z = np.array((sim.F_z[:, par_pl, :].reshape(N_g, N_g)))

    #F_x =
    if i == 0:#Only calculate this the first time
        xx = np.array((sim.x[:, par_pl, :].reshape(N_g, N_g)))
        yy = np.array((sim.y[:, par_pl, :].reshape(N_g, N_g)))
        zz = np.array((sim.z[:, par_pl, :].reshape(N_g, N_g)))


    vImages[i,:] = np.sqrt(vx**2+vy**2+vz**2)[::3,::3]
    v_train[i,:] = np.sqrt(vx**2+vy**2+vz**2)[::3,::3].flatten()

    fImages[i,:] = np.sqrt(F_x**2+F_y**2+F_z**2)[::3,::3]
    f_train[i,:] = np.sqrt(F_x**2+F_y**2+F_z**2)[::3,::3].flatten()

    # fImages[i,:] = np.sqrt(sim.F_x**2+sim.F_y**2+sim.F_z**2)[::3,::3]
    # f_train[i,:] = np.sqrt(sim.F_x**2+sim.F_y**2+sim.F_z**2)[::3,::3].flatten()


    #Approach 1 - Store the proper image
    # vImages[3*i] = vx[::3,::3]
    # vImages[3*i +1] = vy[::3,::3]
    # vImages[3*i+2] = vz[::3,::3]

    # fImages[3*i] = F_x[::3,::3]
    # fImages[3*i +1] = F_y[::3,::3]
    # fImages[3*i+2] = F_z[::3,::3]



    #Approach 2 - Store the flattened Image
    # vFlatImages[3*i] = vx[::3,::3].flatten()
    # vFlatImages[3*i +1] = vy[::3,::3].flatten()
    # vFlatImages[3*i+2] = vz[::3,::3].flatten()


    # fFlatImages[3*i] = F_x[::3,::3].flatten()
    # fFlatImages[3*i +1] = F_y[::3,::3].flatten()
    # fFlatImages[3*i+2] = F_z[::3,::3].flatten()


  0%|          | 0/20000 [00:00<?, ?it/s]

In [63]:
np.savez_compressed(path+'/Data_SD',vTrain=v_train,fTrain=f_train,vTrain_2d = vImages,fTrain_2d=fImages)

In [65]:
# Plot
n_plot = 100
fig = make_subplots(rows=1, cols=2, subplot_titles=('Velocity-Amplitude', 'Force-Amplitude'))

fig.add_heatmap(x=np.arange(0,15),y=np.arange(0,15),z=np.squeeze(vImages[n_plot,:]), row=1, col=1,
                # zmin=0, zmax=1.1,
                showscale=False,
                # zsmooth='best',
                colorscale='Viridis',
                )

fig.add_heatmap(x=np.arange(0,15),y=np.arange(0,15),z=np.squeeze(fImages[n_plot,:]), row=1, col=2,
                # zmin=0, zmax=1.1,
                showscale=True,
                # zsmooth='best',
                colorscale='Viridis',
                )


fig.layout.xaxis1.title = 'X'
fig.layout.yaxis1.title = 'Y'
fig.layout.xaxis2.title = 'X'
fig.layout.yaxis2.title = 'Y'

fig.update_layout(height=525,width=1000)
fig

In [ ]:
xx = np.array((sim.x[:, par_pl, :].reshape(N_g, N_g)))
zz = np.array((sim.z[:, par_pl, :].reshape(N_g, N_g)))
F_x = np.array((sim.F_x[:, par_pl, :].reshape(N_g, N_g)))
F_z = np.array((sim.F_z[:, par_pl, :].reshape(N_g, N_g)))

fig = plt.figure();  ax = fig.add_subplot()

s_i=3
plt.quiver(xx[::s_i,::s_i], zz[::s_i,::s_i], F_x[::s_i,::s_i], F_z[::s_i,::s_i])

# plot the particle
circle1 = plt.Circle((r[0], r[1]), radius=2, color='green'); ax.add_patch(circle1)

In [ ]:
vx, vy, vz = sim.vCalc(a, sigma, r, F)

vx = np.array((sim.vx[:, par_pl, :].reshape(N_g, N_g)))
vz = np.array((sim.vz[:, par_pl, :].reshape(N_g, N_g)))
xx = np.array((sim.x[:, par_pl, :].reshape(N_g, N_g)))
zz = np.array((sim.z[:, par_pl, :].reshape(N_g, N_g)))

fig = plt.figure();  ax = fig.add_subplot()

s_i=3
plt.quiver(xx[::s_i,::s_i], zz[::s_i,::s_i], vx[::s_i,::s_i], vz[::s_i,::s_i])

# plot the particle
circle1 = plt.Circle((r[0], r[1]), radius=2, color='green'); ax.add_patch(circle1)
